In [1]:
import os
import numpy as np
from keras.models import Sequential # To initialise the nn as a sequence of layers
from keras.layers import Convolution2D # To make the convolution layer for 2D images
from keras.layers import MaxPooling2D # 
from keras.layers import GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import CSVLogger
from keras.optimizers import RMSprop
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

csv=CSVLogger("10_encoding.log")
#filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')


# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Convolution2D(32,(2,2),input_shape = (224,224,1),strides=2,name='convo1'))
classifier.add(Convolution2D(64,(3,3), activation = 'relu',name='convo2'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

# Step 1 - Convolution
classifier.add(Convolution2D(64,(3,3),activation = 'relu',name='convo3'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

classifier.add(GlobalAveragePooling2D())
# As our model is still facing the problem so, we need to increase the regulization
classifier.add(Dropout((0.5)))
classifier.add(Dense(20, activation = 'softmax'))

Using TensorFlow backend.


In [2]:

classifier.compile(optimizer = 'adadelta', loss = 'categorical_crossentropy', metrics = ['accuracy'])

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

curr_path = os.getcwd()
basefolder = os.path.dirname(curr_path)

train_folder = os.path.join(basefolder, "Dataset\\Train")
test_folder = os.path.join(basefolder, "Dataset\\dev")
"""
# Changes for linux
train_folder = os.path.join(basefolder, "Dataset/Train")
test_folder = os.path.join(basefolder, "Dataset/dev")
"""
train_set = train_datagen.flow_from_directory(train_folder,target_size=(224, 224),batch_size=64,class_mode='categorical',color_mode='grayscale')

test_set = test_datagen.flow_from_directory(test_folder,target_size=(224, 224),batch_size=64,class_mode='categorical',color_mode='grayscale')

Found 2600 images belonging to 20 classes.
Found 200 images belonging to 20 classes.


In [3]:
history = classifier.fit_generator(train_set,steps_per_epoch=2600,epochs=30,validation_data=test_set,validation_steps=200,callbacks=[csv],verbose=2)

Epoch 1/30
478s - loss: 2.9708 - acc: 0.0681 - val_loss: 2.9234 - val_acc: 0.1050
Epoch 2/30
470s - loss: 2.8475 - acc: 0.1066 - val_loss: 2.8866 - val_acc: 0.1104
Epoch 3/30
470s - loss: 2.7097 - acc: 0.1475 - val_loss: 2.8521 - val_acc: 0.1150
Epoch 4/30
470s - loss: 2.5535 - acc: 0.1926 - val_loss: 2.7670 - val_acc: 0.1552
Epoch 5/30
470s - loss: 2.4025 - acc: 0.2319 - val_loss: 2.7065 - val_acc: 0.1202
Epoch 6/30
470s - loss: 2.2642 - acc: 0.2707 - val_loss: 2.7356 - val_acc: 0.1398
Epoch 7/30
469s - loss: 2.1315 - acc: 0.3090 - val_loss: 2.6402 - val_acc: 0.2103
Epoch 8/30
469s - loss: 2.0082 - acc: 0.3465 - val_loss: 2.6213 - val_acc: 0.2098
Epoch 9/30
470s - loss: 1.9011 - acc: 0.3777 - val_loss: 2.6565 - val_acc: 0.2297
Epoch 10/30
468s - loss: 1.8046 - acc: 0.4063 - val_loss: 2.5816 - val_acc: 0.2196
Epoch 11/30
467s - loss: 1.7092 - acc: 0.4344 - val_loss: 2.6124 - val_acc: 0.2450
Epoch 12/30
467s - loss: 1.6170 - acc: 0.4645 - val_loss: 2.7752 - val_acc: 0.2403
Epoch 13/30
4

In [4]:
classifier.save("10_encoding.h5")